# QF 600 Asset Pricing | YU Lingfeng
## `Homework 4` | Efficient Frontier Revisit

### <font color = green> Activation of necessary libraries. </font>

In [9]:
!pip install matplotlib
!pip install --upgrade openpyxl
!pip install nbconvert

                                              0.0/250.0 kB ? eta -:--:--
     -------------------------------------  245.8/250.0 kB 7.4 MB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.0.10
    Uninstalling openpyxl-3.0.10:
      Successfully uninstalled openpyxl-3.0.10


ERROR: Could not install packages due to an OSError: [WinError 5] 拒绝访问。: 'E:\\software\\anaconda3\\Lib\\site-packages\\~penpyxl\\utils\\cell.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from datetime import datetime

from IPython.display import display
from IPython.display import Image
import random
import warnings

# Ignore all warnings (not recommended for production code)
warnings.filterwarnings("ignore")

In [3]:
def generate_w(n_asset, n_samples):
    col_names = []
    for i in range(n_asset):
        col_names += [f'w{i+1}']
    w = []
    for j in range(n_samples):
        w_rand = []
        for i in range(n_asset):
            w_rand += [random.uniform(0, 1)]

        w += [w_rand] 
    w = pd.DataFrame(w)
    w.columns = col_names
    w['sum'] = w.sum(axis = 1)
    for col in w.iloc[:,:-1].columns:
        w[col] = w[col] / w['sum']
    w['Norm_sum'] = w.iloc[:,:-1].sum(axis = 1) 
    return w.iloc[:, :-2]



In [4]:
def generate_w_reciprocal(n_asset, n_samples):
    col_names = []
    for i in range(n_asset):
        col_names += [f'w{i+1}']
    w = []
    for j in range(n_samples):
        w_rand = []
        for i in range(n_asset):
            w_rand += [1/random.uniform(0, 1)]

        w += [w_rand] 
    w = pd.DataFrame(w)
    w.columns = col_names
    w['sum'] = w.sum(axis = 1)
    for col in w.iloc[:,:-1].columns:
        w[col] = w[col] / w['sum']
    w['Norm_sum'] = w.iloc[:,:-1].sum(axis = 1) 
    return w.iloc[:, :-2]    

In [5]:
t = generate_w_reciprocal(10, 4)
t

,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10
0,0.098411,0.128033,0.271638,0.045974,0.051440,0.092325,0.047181,0.059704,0.070121,0.135174
1,0.062955,0.052854,0.011403,0.011386,0.137484,0.028534,0.011374,0.578458,0.090997,0.014556
2,0.111978,0.008643,0.036487,0.006481,0.778218,0.016234,0.009140,0.005889,0.005722,0.021207
3,0.364593,0.085009,0.256705,0.038701,0.038956,0.036170,0.057922,0.031001,0.059859,0.031084


### <font color = "blue"> `IMPORT` dataset: `Industry_Portfolios.xlsx`. `Market_Portfolio.csv` </font> 
### <font color = "blue"> Validate dataset

> ### Question 1: Minimum-Tracking-Error Frontier

#### Q1.1 minimum-tracking-error frontier generated by the ten industry portfolios.

In [6]:
Ind_R = pd.read_excel("Industry_Portfolios.xlsx", index_col="Date")
Mkt_R = pd.read_excel("Market_Portfolio.xlsx", index_col="Date")

In [7]:
def Min_Tracking_Error_Frontier(Portfolio_R, Market_R, Rf, start, end, step):
    col_names = list(Portfolio_R.columns)
    result = 0
    R_i = Portfolio_R.iloc[:,1:]
    for col in R_i.columns:
        R_i[col] = R_i[col] - Mkt_R.iloc[:,1]
    R_i_matrix = R_i.iloc[:, :].to_numpy()
    
    R_i_mean = R_i.mean(axis = 0)
    R_i_mean = pd.DataFrame(R_i_mean)
    R_i_mean.columns = ['Ri_mean']
    R_i_mean['Ri_std'] = R_i.std(axis = 0)
    R_i_mean['Ri_var'] = R_i_mean['Ri_std'] ** 2

    # Calculate the variance-covariance matrix
    # Setting rowvar=False means that each column represents a variable (asset), 
    # and each row represents an observation (sample).
    V = np.cov(R_i_matrix, rowvar=False)
    # Calculate the V-inverse
    V_inv = np.linalg.inv(V)

    # Calculate the R-prime (transpose)
    R_prime = R_i_mean['Ri_mean'].to_numpy()
    R = np.transpose(R_prime)
    # create the unit vector e
    e_prime = np.ones(len(col_names) - 1)
    e = e_prime.T

    alpha = R_prime @ V_inv @ e
    zeta = R_prime @ V_inv @ R
    delta = np.dot(np.dot(e_prime, V_inv), e)    

    Rmv = alpha / delta
    Rp = np.arange(start, end, step)
    Var = (1 / delta) + (delta / (zeta * delta - alpha ** 2) ) * ((Rp - Rmv) ** 2)
    st_d = Var ** 0.5
    
    frontier_plot = [Rp, st_d]
    
    frontier_plot = pd.DataFrame(frontier_plot)
    frontier_plot.index = ['Y', 'X']

    # name sigma_p for differentiate with previous st_d, for tangency line
    sigma_p = (Rp - Rf) / ((zeta - 2 * alpha * Rf + delta * (Rf**2)) ** 0.5)
      
    # Information Ratio for tangency portfolio = (Rtg - Rf) / sigma_tg
    Info_ratio_tg = (zeta - 2 * alpha * Rf + delta * (Rf**2)) ** 0.5
    
    Rtg = (alpha * Rf - zeta) / (delta * Rf - alpha)
    # Weightage w* = a + b * Rp, on Lecture Notes 2 Page 12
    w_star = ((delta * Rtg - alpha) / (zeta * delta - alpha ** 2)) *\
                                    (V_inv @ R) + ((zeta - alpha * Rtg) / \
                                    (zeta * delta - alpha ** 2)) * (V_inv @ e)    
    w_star = pd.DataFrame(w_star)
    w_star.columns = ['Weightage']
    w_star.index = col_names[1:]
    print(alpha)
    return [frontier_plot.T, sigma_p, V, R_i_mean, Info_ratio_tg, w_star]

In [8]:
MTEF = Min_Tracking_Error_Frontier(Ind_R, Mkt_R, 0, 0, 0.105, 0.005)
MTEF[-1]

IndexError: single positional indexer is out-of-bounds

In [ ]:
information_ratio = MTEF[-2]
weights_tangency = MTEF[-1]

In [ ]:
def plot_frontier(df_coordinates, line_p, x_label, y_label, plt_title,\
                  start, end, step, tan=True):
    partition = (len(df_coordinates)) // 2

    plt.figure(figsize=(8, 6))
    # Create a line plot for y1
    plt.plot(df_coordinates['X'][:partition+1], 
             df_coordinates['Y'][:partition+1], label='Inefficient',
             marker='s', linestyle='--')

    # Create a line plot for y2
    plt.plot(df_coordinates['X'][partition:], 
             df_coordinates['Y'][partition:], label='Efficient Frontier',
             marker='o', linestyle='-')
    

    # Create a line plot for y1
    if tan:
        plt.plot(line_p, np.arange(start, end, step),\
                 label='tangent', linestyle='-') 
    # Add a title
    plt.title(plt_title)
    plt.yticks(np.arange(start, end, step))
    # plt.xlim(0,5.5)
    # Add labels and a legend
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()

    plt.savefig(f'{plt_title}.png')
    # Show the plot
    plt.show()    

    return

In [ ]:
plot_frontier(MTEF[0], MTEF[1], "tracking error", 
              'Expected deviation Return %', 
              'minimum-tracking-error frontier',
             0, 0.105, 0.005, False)

#### Q1.2 minimum-tracking-error frontier with tangency line.

In [ ]:
plot_frontier(MTEF[0], MTEF[1], "tracking error", 
              'Expected deviation Return %', 
              'minimum-tracking-error frontier',
             0, 0.105, 0.005, True)

#### Q1.3 Information Ratio and Tangency porfolio.

In [ ]:
information_ratio

In [ ]:
weights_tangency

> Question 2: Monte Carlo simulation on minimum-variance frontier without short sales

In [ ]:
def plot_scatter(df_coordinates):
    plt.scatter(df_coordinates['X'], df_coordinates['Y'], \
                color='blue', marker='o', s=1)

    # Adding labels and a legend
    plt.xlabel('standard deviation of return')
    plt.ylabel('mean return')
    plt.title('Minimum-Variance Frontier w/o Short Sales')
    plt.legend(labels=[f'{len(df_coordinates):.1e} data points'])

    # Show the plot
    plt.show()    
    return

In [ ]:
def Monte_Carlo_simulation(df_w, R_arr, V_matrix):
    coordinates = pd.DataFrame(index=df_w.index, columns=['X', 'Y'])
    weights_array = df_w.to_numpy()

    coordinates['Y'] = weights_array @ R_arr
    variances = np.einsum('ij,jk,ik->i', weights_array, V, weights_array)
    coordinates['X'] = variances**0.5

    return coordinates

In [ ]:
Rp = Ind_R.iloc[:, 1:].mean(axis = 0).to_numpy()
Rp = Rp.T
matrix = Ind_R.iloc[:, 1:].to_numpy()
# Setting rowvar=False means that each column represents a variable (asset), 
# and each row represents an observation (sample).
V = np.cov(matrix, rowvar=False)

In [ ]:
V

In [ ]:
def Monte_Carlo_simulation_batch(N, n, R_arr, V_matrix, n_assets,\
                                 func=generate_w_reciprocal):
    result = pd.DataFrame(columns=['X', 'Y'])
    if n > 10000:
        n = 10000
    for batch in range(int(N/n)):
        result = pd.concat([result, Monte_Carlo_simulation(func(n_assets, n), \
                                                           R_arr, V_matrix)])
    
    result.index = range(N)
    return result
    

In [ ]:
w_simulation = Monte_Carlo_simulation_batch(100000, 100, Rp, V, 10, generate_w)

In [ ]:
w_simulation.describe()

In [ ]:
w_r_simulation = Monte_Carlo_simulation_batch(100000, 100, Rp, V, 10,\
                                              generate_w_reciprocal)

In [ ]:
w_r_simulation.describe()

#### Q1.1 minimum-tracking-error frontier generated by the ten industry portfolios.

In [ ]:
plot_scatter(w_simulation)

#### Q2.2 Normalised w-reciprocal vector generation of 1e5 data points 

In [ ]:
plot_scatter(w_r_simulation)

## QF 600 Asset Pricing | `Homework 4` | YU Lingfeng
## Efficient Frontier Revisit | Submission 1

> ### Question 1: Minimum-Tracking-Error Frontier

In [ ]:
print(f"The information ratio is {information_ratio}\n")
print("Weightage on tangency portfolio is:")
weights_tangency

#### Q1.1 minimum-tracking-error frontier generated by the ten industry portfolios.

In [ ]:
plot_frontier(MTEF[0], MTEF[1], "tracking error", 
              'Expected deviation Return %', 
              'minimum-tracking-error frontier',
             0, 0.105, 0.005, False)

#### Q1.2 minimum-tracking-error frontier with tangency line.

In [ ]:
plot_frontier(MTEF[0], MTEF[1], "tracking error", 
              'Expected deviation Return %', 
              'minimum-tracking-error frontier',
             0, 0.105, 0.005, True)

> Question 2: Monte Carlo simulation on minimum-variance frontier without short sales

#### Q2.1 Normalised w vector generation of 1e5 data points

In [ ]:
plot_scatter(Monte_Carlo_simulation_batch\
             (100000, 100, Rp, V, 10, generate_w))

#### Q2.2 Normalised w-reciprocal vector generation of 1e5 data points 

In [ ]:
plot_scatter(Monte_Carlo_simulation_batch\
             (100000, 100, Rp, V, 10, generate_w_reciprocal))